### 参考文献

+ [How to Build a Pairs Trading Strategy on Quantopian?](https://www.quantopian.com/posts/how-to-build-a-pairs-trading-strategy-on-quantopian)
+ [CiNii 論文 -  共和分性に基づく最適ペアトレード](https://ci.nii.ac.jp/naid/120006224721/)


# ペアトレーディング

ペアトレーディング（以下ペアトレ）は，マーケットの暴騰や急落といった，急な変化に巻き込まれずに収益を上げていく方法として昔から使われてるストラテジーです。

ここで、似通った性格を持つ銘柄Xと銘柄Yの２つの銘柄があり、長い期間で見ると、どちらの銘柄の価格もいつも大体同じような動きをしているものとしましょう。
短い期間で見てみると、ときには違う動きをすることもあるかもしれませんが、いずれ同じような動きに戻ってくることを期待して、儲けることが出来そうだ，というのがペアトレの基本的な考え方です．どういうことでしょう？

これを少し言い換えてみます。

話を単純にするために、銘柄XとYの価格がほぼ同じくらいであるとします。そうすると、長期的に見て**Xの価格とYの価格の差**はゼロあたりになるはずですが、
ときには、この差が大きくなることがあります。つまり、これは何らかの理由で**Xが大きく値上りしているか、Yが大きく値下がりしている**ということです。
どちらかの銘柄の株価だけ先に動いたのであって、もう片方もいずれ追随していって、この差がまたなくなると考えられるならば、**Xを売りYを買う**というポジションを取り、
そして，価格差がなくなったときに反対売買（Xを買い戻しYを売る）を行えば利益を確定できます。

```python
#（ココに上記の説明をする図を挿入したい）
```


この仕組みは、XとYの間の関係だけに基づいていますから、株式市場全体が勢い良く上昇していたり、下げていたりしていても、原理的には関係ありません。株式市場が勢い良く下げて、Xの価格が下がるときにはYの価格も下がるでしょうから、これらの２つの動きは相殺されることになります。

一般的に，株式の価格はランダムウォーク（酔っぱらいのオジサン歩き＝どっちに動くか分からない）に従っており、予測することは出来ないと考えられます。
しかし、２つの銘柄の株価の動きの違いに着目すると、組み合わせ次第では、一定の値の周りで変動するという現象がしばしば観測されます。
これを平均回帰性といいますが、そのような銘柄の組み合わせを見つけて、ペアトレを実際にシミュレーションしていきましょう。

このノートでは，Quantopian Research を使ってどのようにペアトレを構築していくかを学んで行きます．ノート修了時には，

+ ペアトレ構築方法
+ 銘柄ペアの探し方
+ 銘柄ペアの評価方法
+ 共和分とは何か
+ どのように共和分をテストするか
+ どのように取引シグナルを作るか
+ python library pandas の機能の一部
+ matplotlib の機能の一部

を理解していることを目標にします．

## 平均回帰性

ペアトレでは<u>２つの銘柄の価格の差がだいたい一定の範囲に収まっていること</u>が必要です。
いったん価格の差が開いても、元に戻る性質のことを、**平均回帰性**と言います。

>メモ：もし、片方の銘柄だけ値上がりし続けたり、値下がりし続ける場合には、価格の差はどんどん広がって、大きな損をすることもあり得ます

例を見てみましょう．
金採掘会社関連銘柄のGDXとABXについて、2010/1/3から2017/12/31までの動きを見てみます。
それぞれの株価のレベルが違うので、移動平均を取りながら価格の比を計算し、調整して差を取り、そしてその差の平均をプロットしてみました。


In [ ]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt

def draw(x, y, term, start_date, end_date):
    df = get_pricing([x, y], fields='price', start_date=start_date, end_date=end_date)
    return draw_df(df, x, y, term)
    
def draw_df(df, x, y, term):
    beta = df[x].rolling(window=term).mean()/df[y].rolling(window=term).mean()
    df['spread'] = df[x] - beta*df[y]
    df['spread mean'] = df["spread"].mean()
    df.plot()
    return df
    
GDX = symbols('GDX')
ABX = symbols('ABX')
term = 60
start_date='2010-01-03'
end_date='2017-12-31'
df_GDX_ABX = draw(GDX, ABX, term, start_date, end_date)

赤線が２つの銘柄の差の動きです。
ゼロの線を取り巻くように上下に動いていますね。

なんかもう儲かる気がしませんか（笑）


## ランダム・ウォークする株価

本当に儲かる可能性のある銘柄のペアなのか、そうでないのかをチェックする方法があります。
チェックして、大丈夫そうでも、実際には上手く行かないかもしれませんが、チェックして駄目なら、実際に駄目な可能性はぐっと高くなります。

ここで、そのチェック方法を見ていきましょう。

株価はランダムウォークするはずですから、まずは仮想の株価を２つ、シミュレーションで作ってみます。



In [ ]:
def make_price_movement(init_price, average_return, volatility, num_units):
    returns = np.random.normal(average_return/num_units, volatility/math.sqrt(num_units), num_units)
    price_list = [init_price]
    for x in returns:
        price_list.append(price_list[-1]*(1.0+x))
    return price_list

INIT_PRICE = 10.0
AVERAGE_RETURN = 0.05
VOLATILITY = 0.2
NUM_UNITS = 252 
np.random.seed(10)

price_movement1 = make_price_movement(INIT_PRICE, AVERAGE_RETURN, VOLATILITY, NUM_UNITS)
price_movement2 = make_price_movement(INIT_PRICE, AVERAGE_RETURN, VOLATILITY, NUM_UNITS)
df = pd.DataFrame({'random_x':price_movement1, 'random_y':price_movement2})
df_randoms = draw_df(df, 'random_x', 'random_y', term)


random_x と random_y は同じように動いているようには見えませんね．しかし，赤い線のspreadは，紫の線 spread mean の周りを動いているように見えます．

**お互い関係のない乱数を2つ作っただけ**なのに，平均回帰性があるように見えるということは，どういうことでしょう？

ペアトレは過去に観測されたspread同様，未来も同じように大きな差が生まれたら収束してくれる事を期待できないと，トレードできません．乱数で作ったペアでも平均回帰性が出るようでは，ペア探しに支障をきたしてしまいます．一体どうしたらよいのでしょう？



## 共和分

金鉱関連株同士のGDXとABXのように、実際に同じような性格をもって動くペアと、たまたま同じように動いて見えることがあるだけの、乱数で作ったペアの良し悪しを見分け、騙されないようにするには、どうしたら良いでしょうか。

そのための分析方法として、**共和分**というものがあります。
ここで、共和分を使って、確かめてみましょう．



### 共和分をテストする

python の統計モデルライブラリ [StatsModels](http://www.statsmodels.org/dev/index.html) の `statsmodels.tsa.stattools`の中にある [coint](http://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.coint.html) を使って、２つの時系列が共和分しているかどうか確認します。

`coint(X,Y)` は３つの値 coint_t, p_value, crit_value を返します．

そのなかの，p_value (p値)が小さければ小さいほど，2つの価格が共和分性を持つことを示します．
一般的に、p値が0.05を下回っていれば、共和分性があると言って良いと言われています。


In [ ]:
from statsmodels.tsa.stattools import coint
# GDXとABXのケース
coint_t, p_value, crit_value = coint(df_GDX_ABX[GDX], df_GDX_ABX[ABX])
print 'GDXとADX:', p_value

# 乱数のケース
coint_t, p_value, crit_value = coint(price_movement1, price_movement2)
print '２つの乱数:', p_value

大きな違いが出ましたね。

### 関連のありそうな銘柄とは

それでは、共和分する銘柄ペアを探すには、どうしたら良いでしょうか。

では，<font color=red>相場にある全銘柄でペアを作り，片っ端からその差の共和分を取得し，良いペアを探してみましょう!</font>
とやりたいところですが，さすがに大変です。

一般的に同じような値動きをする銘柄は，同じ業種の銘柄や同じ商品のサプライヤー等，経済的に関連がありそうな銘柄が似たような値動きをします（例：資生堂と花王（化粧品）や，村田製作所とアルプス電気（iphoneサプライヤ））。そこで，関連ありそうな銘柄を複数集めて，そこで総当りさせるコードを書いてみましょう。

Quantopianでは，アメリカ株だけ取得可能ですので，ここでは金鉱株を集めて共和分を見ていきたいと思います．


In [ ]:
import itertools

def find_cointegrated_pairs(pair_list, start_date, end_date):
    prices = get_pricing(pair_list, start_date=start_date, end_date=end_date, fields="price", frequency="daily")
    prices.columns = map(lambda x: x.symbol, prices.columns)
    pairs = list(itertools.combinations(pair_list, 2))
    p_value_list = list()
    for x, y in pairs:
        _, p_value, _ = coint(prices[x], prices[y])
        p_value_list.append([x, y, p_value])
    return sorted(p_value_list, key=lambda x: x[2])  

start_date = "2011-01-01"
end_date = "2018-01-01"
pairlist = ['AEM', 'GG', 'AUY', 'KGC', 'EGO', 'ABX', 'NEM', 'GDX']
find_cointegrated_pairs(pairlist, start_date, end_date)

p値が0.05を下回っている良さそうな組み合わせがありますね。さきほどのGDXとABXよりも、[ゴールドコープ【GG】](https://stocks.finance.yahoo.co.jp/us/detail/GG)と[エルドラド・ゴールド【EGO】](https://stocks.finance.yahoo.co.jp/us/detail/EGO)の方が良い値ですので、こちらに注目してみることにします。
なお、違う期間を取って組み合わせを評価すれば、また違った結果が出てきますので、この組み合わせが一番いいとは限らないことに注意してください。

In [ ]:
GG = symbols('GG')
EGO = symbols('EGO')
term = 60
start_date='2010-01-03'
end_date='2017-12-31'
df_GG_EGO = draw(GG, EGO, term, start_date, end_date)

良さそうな感じですね。
では，この2つの銘柄を2011年から2017年末まで取得して，実際のシュミュレーションを行ってみましょう．


（対比用）

In [ ]:
GG = symbols('GG')
NEM = symbols('NEM')
term = 60
start_date='2010-01-03'
end_date='2017-12-31'
df_GG_NEM = draw(GG, NEM, term, start_date, end_date)

赤線は、移動平均からの価格差ですが、これが３以上なら、GGを売りEGOを買い，−３以下であればEGOを売りGGを買うというストラテジーを組んでみましょう。

```python
#質問-10から10まで横線を引く方法はないでしょうかね？
```

実際に取引するときには、取引の費用や、株の流動性なども問題になりますが、そういう点については、QuantopianのAlgorithmの機能が役に立ちます。ここでは、Algorithmで作業をする前に、ざっくりとした計算でストラテジーの様子を把握します。

In [ ]:
x = symbols('GG')
y = symbols('EGO')
term = 60
start_date='2010-01-03'
end_date='2017-12-31'
threshold = 3

# 価格取得
df = get_pricing([x, y], fields='price', start_date=start_date, end_date=end_date)

# スプレッド計算
beta = df[x].rolling(window=term).mean()/df[y].rolling(window=term).mean()
df['spread'] = df[x] - beta*df[y]

# 一定の値を超えたかどうかの判定
df['above_threshold'] = df["spread"] > threshold
df['below_threshold'] = df["spread"] < -threshold

# ポジション保有判定
df['position'] = 0
# for i, row in df.iterrows():
#     if row['above_threshold']:  # 閾値を超えたら取引開始
#         row['position'] = 1
#     elif row['below_threshold']:  # 閾値を下回ったら反対取引開始
#         row['position'] = -1
#     elif i > term:  # 移動平均計算期間は取引しない前提
#         row['position'] = df['position'].loc[i-1] # 前の日の状態と同じにする
df.loc[df['above_threshold'], 'position'] = 1
df.loc[df['below_threshold'], 'position'] = -1


        
# 毎日至近の補充をする前提でざっくり利益の計算をする
df["x_return"] = df[x].pct_change().shift(-1)
df["y_return"] = df[y].pct_change().shift(-1)

# 移動平均計算期間は取引しない前提
df = df.iloc[term:]    

df['pair_return'] = (df['x_return']-df['y_return']) * df['position']
df['amount'] = (df['pair_return']+1.0).prod()

In [ ]:
spy = get_pricing("SPY", start_date=start_date, end_date=end_date, fields='price', frequency='daily')    

In [ ]:
df["amount"].plot()
spy.pct_change().cumprod().plot()


うーーーーん・・・2015年以降はまあまあいいのですが，変動が激しくツライですし，これではSPYを黙って持っている方が全然ましということになります．

と，なかなか人生うまく行かないわけですが，せっかくですので，みんなで手分けして良いストラテジーを探してみましょう．

思いついた改良ポイントを紹介しますので，これをヒントに改良してみてもらってもよいですし，何かご自身で思いつくモノがあればやってみて下さい．

(๑•̀ㅂ•́)و✧

### 改良（？）ヒント

+ 違うペアを探す
+ 違う業種を探す [S&P 500 Map](https://finviz.com/map.ashx?t=sec)
+ 複数ペアの組み合わせ
+ term をもっと短く/長く
+ 20日移動平均+ボリンジャーバンドの組み合わせ

![](https://4.bp.blogspot.com/-zTvzECyWEsk/VwIjHWMdszI/AAAAAAAA5e4/W_kAnVythXoHGzGO3AkgrHImS3cpvMiuQ/s330/internet_kanki_man1.png)

まあ人生そんなに上手く行くことはありません．


## このあとは

アルゴリズムでシミュレーションテンプレートを作っておく？
